Less of a unit test, more an exploration of waffles.

In [735]:
filename = 'samples/waffle.tex'

In [736]:
# debug
from importlib import reload
reload(TexSoup)

import TexSoup
import re
import numpy as np

In [737]:
with open(filename,'r') as f:
    tex_doc = f.read()

Sometimes there are weird errors like spaces in begins/ends -- fix these first:

In [738]:
from TexSoup import preprocessing
reload(preprocessing)

<module 'TexSoup.preprocessing' from '/Users/jnaiman/TexSoup/TexSoup/preprocessing.py'>

In [739]:
tex_doc = preprocessing.process_begin_end(tex_doc)

In [740]:
newcommands, newenvironments = preprocessing.get_newcommands_and_newenvs(tex_doc,
                                                                         verbose=True)

In [741]:
newcommands

[('\\be', '\\newcommand{\\be}{\\begin{equation}}', 34, 68),
 ('\\ee', '\\newcommand{\\ee}{\\end{equation}}', 69, 101),
 ('\\bee', '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}', 103, 153),
 ('\\eee', '\\newcommand{\\eee}{\\end{equation}}', 154, 187),
 ('\\beee',
  '\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}',
  189,
  243),
 ('\\eeee', '\\newcommand{\\eeee}{\\end{equation}}', 244, 278),
 ('\\etal ', '\\def \\etal {{\\em et al.}}', 281, 306)]

In [742]:
newenvironments

[]

In [743]:
reload(preprocessing)

<module 'TexSoup.preprocessing' from '/Users/jnaiman/TexSoup/TexSoup/preprocessing.py'>

In [744]:
args_newcommands = preprocessing.find_args_newcommands(newcommands,error_out=True, verbose=True)

In [745]:
args_newcommands

[['\\be', '\\newcommand{\\be}{\\begin{equation}}', 34, 68, 0],
 ['\\ee', '\\newcommand{\\ee}{\\end{equation}}', 69, 101, 0],
 ['\\bee',
  '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}',
  103,
  153,
  1],
 ['\\eee', '\\newcommand{\\eee}{\\end{equation}}', 154, 187, 0],
 ['\\beee',
  '\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}',
  189,
  243,
  2],
 ['\\eeee', '\\newcommand{\\eeee}{\\end{equation}}', 244, 278, 0],
 ['\\etal ', '\\def \\etal {{\\em et al.}}', 281, 306, 0]]

In [746]:
args_newenvironments = preprocessing.find_args_newenvironments(newenvironments,
                                                               error_out=True, verbose=True)

In [747]:
args_newenvironments

[]

In [748]:
# go through and find if there is begin/end in there
# note: at this point all beginnings/endings should be fixed
find_replace = []
comments = []
for ic,nc in enumerate(args_newcommands):
    if nc == 'error':
        continue
    n,fn,i1,i2,nArgs = nc
    if nArgs == 0: # no arguements
        if '\\begin' in fn and not ('\\end' in fn):
            i = fn.index(n+'}') + len(n+'}')
            ind1,ind2,err = spc(fn[i:],
                            function='',dopen='{',
                            dclose='}',
                           error_out=False)
            if not err:
                cmd = fn[i:][ind1+1:ind2-1]
                find_replace.append((n,cmd,nArgs))
            comments.append(fn)

        elif '\\end' in fn and not ('\\begin' in fn):
            i = fn.index(n+'}') + len(n+'}')
            ind1,ind2,err = spc(fn[i:],
                            function='',dopen='{',
                            dclose='}',
                           error_out=False)
            if not err:
                cmd = fn[i:][ind1+1:ind2-1]
                find_replace.append((n,cmd,nArgs))
            comments.append(fn)
        elif '\\def' in fn: # def statement
            pass
        else: # have both
            print('have both')
            import sys; sys.exit()
    else:
        find_replace.append((n,fn,nArgs))
        comments.append(fn)

In [749]:
find_replace

[('\\be', '\\begin{equation}', 0),
 ('\\ee', '\\end{equation}', 0),
 ('\\bee', '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}', 1),
 ('\\eee', '\\end{equation}', 0),
 ('\\beee', '\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}', 2),
 ('\\eeee', '\\end{equation}', 0)]

In [750]:
comments

['\\newcommand{\\be}{\\begin{equation}}',
 '\\newcommand{\\ee}{\\end{equation}}',
 '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}',
 '\\newcommand{\\eee}{\\end{equation}}',
 '\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}',
 '\\newcommand{\\eeee}{\\end{equation}}']

In [751]:
#print(tex_doc)

In [815]:
text = tex_doc
verbose = True

icount = 0
# find/replace -- require a whitespace after
for instr, outstr,nArgs in find_replace:
    if nArgs == 0: 
        ind = 0
        text_out = []
        search_cmd = re.escape(instr) + '(\\s{1,})'

        while ind < len(text):
            if re.search(search_cmd, text[ind:]):
                istart,iend1 = re.search(search_cmd, text[ind:]).span()
                # get the command specifically
                _,iend = re.search(re.escape(instr),text[ind:][istart:iend1]).span()
                #iend += iend1
                i1 = istart
                i2 = istart+iend
                # make sure we are not in a command
                inCommand = False
                for n,fn,i1c,i2c in newcommands:
                    if i1+ind >= i1c and i2+ind <= i2c: # inside
                        inCommand = True

                if not inCommand: # not in a command
                    text_out.append(text[ind:][:istart])
                    text_out.append(outstr)
                    ind += i2
                else: # in command, move on
                    ind += i2
            else: # not in there anymore
                text_out.append(text[ind:])
                ind += len(text[ind:])

        text = "".join(text_out)
    else: # has arguments -- need to parse these as well for replacement
        ind = 0
        text_out = []
        search_cmd = re.escape(instr) + r'(\s*){' # include bracket in search
        while ind < len(text):
            if re.search(search_cmd, text[ind:]): # start of command
                istart,iend = re.search(search_cmd, text[ind:]).span()
                args = {} # get all arguments and store their values
                err = False
                #if icount > 0: import sys; sys.exit()
                for ia in range(nArgs): # loop through all expected arguments
                    # search for matching brakets, starting at starting {
                    ind1,ind2,err = spc(text[ind+iend-1:],
                                    function='',dopen='{',
                                    dclose='}',
                                   error_out=False)
                    if not err: # found it!
                        args[ia+1] = text[ind:][iend-1:][ind1+1:ind2-1]
                        # now we have to update everything for the next look at args
                        iend += ind2
                    else: # have an issue, carry on
                        if verbose: print('have issue finding {} for replacement args')
                        iend += 1
                        break
                    #import sys; sys.exit()
                    #if ia>0: import sys; sys.exit()
                #import sys; sys.exit()
                # replace
                outstr_mod = outstr
                for k,v in args.items():
                    outstr_mod = outstr_mod.replace('#'+str(k), v)
                # now get just inside part
                ind1,ind2,err = spc(outstr_mod[::-1],
                    function='',dopen='}',
                    dclose='{',
                   error_out=False)
                outstr_mod = outstr_mod[::-1][ind1+1:ind2-1][::-1]
                #if nArgs > 1: import sys; sys.exit()
                
                inCommand = False
                i1 = istart
                i2 = iend-1
                for n,fn,i1c,i2c in newcommands:
                    if i1+ind >= i1c and i2+ind <= i2c: # inside
                        inCommand = True
                        
                if not inCommand:      
                    text_out.append(text[ind:][:istart])
                    text_out.append(outstr_mod)
                ind += iend-1
                if verbose: print(args)
                #import sys; sys.exit()
            else:
                text_out.append(text[ind:])
                ind += len(text[ind:])
        text = "".join(text_out)
        icount += 1


{1: 'lenseq'}
{1: 'lenseq2', 2: 'A(x)=5'}


In [816]:
print(text)

\documentclass[a4paper]{article}

\newcommand{\be}{\begin{equation}}
\newcommand{\ee}{\end{equation}}

\newcommand{\bee}[1]{\begin{equation}\label{e:#1}}
\newcommand{\eee}{\end{equation}}

\newcommand{\beee}[2]{\begin{equation}\label{e:#1} #2}
\newcommand{\eeee}{\end{equation}}


\def \etal {{\em et al.}}

\begin{document}

\section{Waffles waffles waffles}

\subsection{Waffles are cooler than you}

Waffles have significantly more pockets than you.  

\[
P(W) > P(\rm{you})
\]

Ok, maybe you are at least as cool as waffles:

$$
C_{ool}(\rm{you}) \ge C_{ool}(W)
$$

Sometimes though, you might type things in silly ways $C_ {ool  } (\rm{anybody}) >   0$.

\begin{equation}
C_{   ool} \ne 0
\end{equation}

Also, let's do a silly redefined an equation:
\begin{equation}
C = C_2 + 1
\end{equation}

Also also, just one more time:
\begin{equation}
C = C_2 + 1
\end{equation}


Sometimes I like to write with weird commands like Naiman \etal\ which can make it tricky to parse things.

\begin{equatio

In [806]:
err

False

In [792]:
print(''.join(text_out))


In [777]:
outstr_mod[::-1][ind1+1:ind2-1][::-1]

'\\begin{equation}\\label{e:lenseq}'

In [653]:
text[ind+iend-1:]

'\\eeee\n\nGetting some refs in \\ref{lenseq} and \\ref{lenseq2}.\n\n\n\\end{document}\n\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:lenseq}}    A(\\xi) = \\frac12\\left(\\xi+\\frac1\\xi\\right),\n\\end{equation}\n\n\\beee{lenseq2}{A(x)=5}\\eeee\n\nGetting some refs in \\ref{lenseq} and \\ref{lenseq2}.\n\n\n\\end{document}\n'

In [525]:
outstr_mod = outstr
for k,v in args.items():
    outstr_mod = outstr_mod.replace('#'+str(k), v)

In [528]:
instr

'\\beee'

In [527]:
outstr_mod

'\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:lenseq2} A(x)=5}'

In [365]:
comments

['\\newcommand{\\be}{\\begin{equation}}',
 '\\newcommand{\\ee}{\\end{equation}}']

In [366]:
find_replace

[('\\be', '\\begin{equation}'), ('\\ee', '\\end{equation}')]

In [352]:
args

[['\\be', '\\newcommand{\\be}{\\begin{equation}}', 34, 68, 0],
 ['\\ee', '\\newcommand{\\ee}{\\end{equation}}', 69, 101, 0],
 ['\\bee',
  '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}',
  103,
  153,
  1],
 ['\\eee', '\\newcommand{\\eee}{\\end{equation}}', 154, 187, 0],
 ['\\etal ', '\\def \\etal {{\\em et al.}}', 190, 215, 0]]

In [347]:
# go through and find if there is begin/end in there
# note: at this point all beginnings/endings should be fixed
find_replace = []
comments = []
for ic,nc in enumerate(newcommands):
    if nc == 'error':
        continue
    n,fn,i1,i2 = nc
    if '#' not in fn:
        if '\\begin' in fn and not ('\\end' in fn):
            i = fn.index(n+'}') + len(n+'}')
            ind1,ind2,err = spc(fn[i:],
                            function='',dopen='{',
                            dclose='}',
                           error_out=False)
            if not err:
                cmd = fn[i:][ind1+1:ind2-1]
                find_replace.append((n,cmd))
            comments.append(fn)

        elif '\\end' in fn and not ('\\begin' in fn):
            i = fn.index(n+'}') + len(n+'}')
            ind1,ind2,err = spc(fn[i:],
                            function='',dopen='{',
                            dclose='}',
                           error_out=False)
            if not err:
                cmd = fn[i:][ind1+1:ind2-1]
                find_replace.append((n,cmd))
            comments.append(fn)
        elif '\\def' in fn: # def statement
            pass
        else: # have both
            print('have both')
            import sys; sys.exit()
    else:
        import sys; sys.exit()
    
for ic,nc in enumerate(newenvironments):
    if nc == 'error':
        continue
    n,fn,i1,i2 = nc
    import sys; sys.exit()
    if '\\begin' in fn and not ('\\end' in fn):
        i = fn.index(n+'}') + len(n+'}')
        ind1,ind2,err = spc(fn[i:],
                        function='',dopen='{',
                        dclose='}',
                       error_out=False)
        if not err:
            cmd = fn[i:][ind1+1:ind2-1]
            find_replace.append((n,cmd))
        comments.append(fn)
            
    elif '\\end' in fn and not ('\\begin' in fn):
        i = fn.index(n+'}') + len(n+'}')
        ind1,ind2,err = spc(fn[i:],
                        function='',dopen='{',
                        dclose='}',
                       error_out=False)
        if not err:
            cmd = fn[i:][ind1+1:ind2-1]
            find_replace.append((n,cmd))
        comments.append(fn)
        
    elif '\\def' in fn: # def statement
        pass
    else: # have both
        print('have both')
        import sys; sys.exit()

        
# find/replace -- require a whitespace after
for instr, outstr in find_replace:
    if '#' not in outstr: 
        ind = 0
        text_out = []
        search_cmd = re.escape(instr) + '(\\s{1,})'

        while ind < len(text):
            if re.search(search_cmd, text[ind:]):
                istart,iend1 = re.search(search_cmd, text[ind:]).span()
                # get the command specifically
                _,iend = re.search(re.escape(instr),text[ind:][istart:iend1]).span()
                #iend += iend1
                i1 = istart
                i2 = istart+iend
                # make sure we are not in a command
                inCommand = False
                for n,fn,i1c,i2c in newcommands:
                    if i1+ind >= i1c and i2+ind <= i2c: # inside
                        inCommand = True

                if not inCommand: # not in a command
                    text_out.append(text[ind:][:istart])
                    text_out.append(outstr)
                    ind += i2
                else: # in command, move on
                    ind += i2
            else: # not in there anymore
                text_out.append(text[ind:])
                ind += len(text[ind:])

        text = "".join(text_out)
    else:
        import sys; sys.exit()
    
# finally, comment out the new commands for the soup
text_out = text
for fn in comments:
    fnout = '%' + fn
    text_out = text_out.replace(fn,fnout)
text = text_out
    
tex_doc = text

SystemExit: 

In [308]:
print(tex_doc)

\documentclass[a4paper]{article}

%\newcommand{\be}{\begin{equation}}
%\newcommand{\ee}{\end{equation}}

\def \etal {{\em et al.}}

\begin{document}

\section{Waffles waffles waffles}

\subsection{Waffles are cooler than you}

Waffles have significantly more pockets than you.  

\[
P(W) > P(\rm{you})
\]

Ok, maybe you are at least as cool as waffles:

$$
C_{ool}(\rm{you}) \ge C_{ool}(W)
$$

Sometimes though, you might type things in silly ways $C_ {ool  } (\rm{anybody}) >   0$.

\begin{equation}
C_{   ool} \ne 0
\end{equation}

Also, let's do a silly redefined an equation:
\begin{equation}
C = C_2 + 1
\end{equation}

Also also, just one more time:
\begin{equation}
C = C_2 + 1
\end{equation}


Sometimes I like to write with weird commands like Naiman \etal\ which can make it tricky to parse things.

\end{document}



In [309]:
soup = TexSoup.TexSoup(r''+tex_doc, tolerance=0)

In [310]:
for iss,s in enumerate(soup.all):
    print('----------- soup element =', iss, '---------------')
    print(type(s))
    print(type(s.expr))
    print(s)
    print('----------------')

----------- soup element = 0 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexCmd'>
\documentclass[a4paper]{article}
----------------
----------- soup element = 1 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexText'>


----------------
----------- soup element = 2 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexText'>


----------------
----------- soup element = 3 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexText'>
%\newcommand{\be}{\begin{equation}}
----------------
----------- soup element = 4 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexText'>


----------------
----------- soup element = 5 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexText'>
%\newcommand{\ee}{\end{equation}}
----------------
----------- soup element = 6 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexText'>


----------------
----------- soup eleme

In [311]:
for iss,s in enumerate(soup.all):
    print('----------- soup element =', iss, '---------------')
    if type(s.expr) == TexSoup.data.TexNamedEnv: # parts of the text
        if 'begin' in s.expr.begin and 'document' in s.expr.begin:
            for isss,ss in enumerate(s.all):
                print('----------- sub-soup element =', isss, '---------------')
                #print(type(ss))
                print(type(ss.expr))
                print(ss)
                print('-----------------')
        else:
            print(' ******** not in document *********** ')

----------- soup element = 0 ---------------
----------- soup element = 1 ---------------
----------- soup element = 2 ---------------
----------- soup element = 3 ---------------
----------- soup element = 4 ---------------
----------- soup element = 5 ---------------
----------- soup element = 6 ---------------
----------- soup element = 7 ---------------
----------- soup element = 8 ---------------
----------- soup element = 9 ---------------
----------- soup element = 10 ---------------
----------- soup element = 11 ---------------
----------- sub-soup element = 0 ---------------
<class 'TexSoup.data.TexText'>


-----------------
----------- sub-soup element = 1 ---------------
<class 'TexSoup.data.TexText'>


-----------------
----------- sub-soup element = 2 ---------------
<class 'TexSoup.data.TexCmd'>
\section{Waffles waffles waffles}
-----------------
----------- sub-soup element = 3 ---------------
<class 'TexSoup.data.TexText'>


-----------------
----------- sub-soup elemen